### Noelle Landauer
CIS 277A, Lab 6, 11/8/22

## Bumblebee species in Oregon

Original sources of [2018](https://www.gbif.org/dataset/b2974853-6c41-4c63-a11b-7989e58a3ad4) and [2019](https://www.gbif.org/dataset/3301620d-6750-434e-97ad-abfac165bb9c) data

To keep the imported files to more managable size, this week I will focus on Oregon's bumblebees (genus Bombus). I have two pools of data to combine together, from 2018 and 2019. To simplfy data cleanup, I eliminated unneeded columns and filtered rows to only include genus Bombus in the original Excel files. First, we need to import each dataset into a Dataframe.

In [1]:
import pandas as pd
import pyodbc
import seaborn as sns

In [2]:
bombus18 = pd.read_excel(r"C:\Users\noell\OneDrive\Documents\Classes\CIS277A_DataAnalytics\277A_Lab6_NoelleLandauer\Bombus2018.xlsx")
bombus19 = pd.read_excel(r"C:\Users\noell\OneDrive\Documents\Classes\CIS277A_DataAnalytics\277A_Lab6_NoelleLandauer\Bombus2019.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\noell\\OneDrive\\Documents\\Classes\\CIS277A_DataAnalytics\\277A_Lab6_NoelleLandauer\\Bombus2018.xlsx'

Let's take a look at the columns of each Dataframe to make sure they are reasonably aligned for merging:

In [ ]:
bombus18.info()
bombus19.info()

The longitude, latitude, and day/month/year have datatype float in 2018, int in 2019. This will resolve to float when the columns merge, so I'm not going to worry about it now. Otherwise the columns between the two Dataframes look compatible.

Since each row in both Dataframes represents a separate bee, we want all the rows of one dataset added to the bottom of the other dataset, rather than joining rows. So we will concantenate the two Dataframes:

In [ ]:
all_bombus = pd.concat([bombus18, bombus19], ignore_index=True)

In [ ]:
all_bombus

By scrolling over the dates. I can see that the 2018 rows are at the top, and 2019 at the bottom. I can also see a few rows at the top where the species has not been determined, so let's see how many:

In [ ]:
missing_count = all_bombus.shape[0] - all_bombus.count()
print(missing_count)

Since there are only 17 missing species determinations and 1 missing date out of nearly 5000 rows, I'm going to remove them from the set: 

In [ ]:
all_bombus = all_bombus.dropna()
missing_count = all_bombus.shape[0] - all_bombus.count()
print(missing_count)

Next let's reset the index. There are two columns with unique identifying rows to potentially use as a key. One is the GBIF database identifier (gbifID), the other is the Oregon State Arthropod Collection url (occurenceID), which contains either an OBA or OSAC number embedded in a string. Since the occurenceID is more complex than is necessary for this task, I'll use the gbifID for the Dataframe index.

In [ ]:
all_bombus.set_index('gbifID', inplace=True)

Next I want to take a look at the data grouped by species and year:

In [ ]:
bombus_species_by_year = pd.crosstab(all_bombus['species'],all_bombus['year'].astype('int'))
bombus_species_by_year

A handful of bees were apparently collected in 2017, but since they have location and date data I'll leave them for now. More concerning are values of 0 under B. bifarius (2019) and B. vancouverensis (2018), which is a bit strange for relatively numerous bees. A little bit of internet [sleuthing](https://washingtonbumblebees.org/bumblebee-field-id/no-red-yes-stripes/two-form-bumble-bee-bombus-bifarius/) reveals that the species identification of B. bifarius west of the Rockies has been hotly debately for years, as it has distinct coloration compared to its eastern counterparts and similarities to B. vancouverensis, found on its namesake island. A recent genetic analysis found gene flow between the bifarius and vancouverensis populations, so now all western bifarius bees are being [rolled over](https://bugguide.net/node/view/184373) into vancouverensis. (Taxonomy!) So I will update the all_bombus and bombus_species_by_year Dataframes to reflect this change.

In [ ]:
all_bombus = all_bombus.replace(to_replace='Bombus bifarius', value='Bombus vancouverensis')
bombus_species_by_year = pd.crosstab(all_bombus['species'],all_bombus['year'].astype('int'))
bombus_species_by_year

Finally, let's get a total count of each species in Oregon to see the both the rarest and most common species. Using this data for population numbers is a bit tricky, as the Oregon Bee Atlas data is biased towards diversity and rarer bees. The more common species are not collected in proportion to their (large) numbers; instead a few are caught at each site to indicate their presence, not a absolute count. But we can still get an idea that some species are extremely widespread while others are rare.

In [ ]:
bombus_species = all_bombus['species'].value_counts()
bombus_species

Same data, plotted in a bar graph:

In [ ]:
bombus_species.plot.barh(
    title='Bumblebee species frequency in the Oregon Bee Atlas')

## Have IMDB ratings changed over time?

Connection object:

In [ ]:
connection = pyodbc.connect(
    server="cisdbss.pcc.edu",
    user="275student",
    password="275student",
    database="IMDB",
    driver="{ODBC Driver 17 for SQL Server}")

connection

SQL query to retrieve all movies with at least 5000 votes and their ratings from the IMDB database 

In [ ]:
sql = '''SELECT primaryTitle, tb.tconst, startYear, averageRating, numVotes
FROM title_basics tb
JOIN title_ratings tr ON tb.tconst=tr.tconst
WHERE titleType = 'movie'
AND numVotes > 5000
AND isAdult = 0;'''

Query database and place in a Dataframe:

In [ ]:
movie_ratings = pd.read_sql_query(sql, connection)
movie_ratings

Set the index to tconst, since it was already conveniently set up as a key in the database:

In [ ]:
movie_ratings.set_index('tconst', inplace=True)
movie_ratings.index

Let's look at average rating per year to start:

In [ ]:
avg_rating_by_year = movie_ratings.groupby('startYear').mean()
avg_rating_by_year

Plot over time:

In [ ]:
year_ratings_plot = sns.lineplot(data=avg_rating_by_year, x='startYear', y='averageRating')
year_ratings_plot.annotate(text='IMDB founded', 
                           xy=(1990, 6.6), 
                           xytext=(1991, 6.8), 
                           arrowprops=dict(facecolor='red', width=3, headwidth=12, headlength=6)
                          )
year_ratings_plot.set(title='Average movie rating per year')

We could hypothesize that only more well-known movies prior to IMDB's founding in 1990 receive more than 5000 votes, and this biases the older movie's ratings. How many movies received more than 5000 votes per year?

In [ ]:
movie_count_by_year = movie_ratings.groupby('startYear').count()
count_plot = sns.lineplot(data=movie_count_by_year, x='startYear', y='numVotes')
count_plot.set(title='Number of movies with at least 5000 votes per year')

And are the number of votes changing over time as well?

In [ ]:
numvotes_plot = sns.lineplot(data=avg_rating_by_year, x='startYear', y='numVotes')
numvotes_plot.set(title='Average number of votes per year (minimum 5000)')

It looks like the number of votes are increasing, but there is a lot of fluctuation from year to year. Perhaps if we create bins for years and the number of votes, the pattern may be clearer. The year groupings are a bit arbitrary, but I wanted to capture the three phases of the "average movie rating per year" graph: high early ratings, dropoff in the 70s-80s, post-IMDB. Since numVotes has a lot of variation, with a few movies on the tail end with very high number of votes, I used quintiles for these bins.

In [ ]:
year_bins = [1915, 1960, 1990, 2018]
movie_ratings['YearGroup'] = pd.cut(movie_ratings['startYear'], 
                                    bins=year_bins, 
                                    labels=['Premodern', 'Early Modern', 'IMDB Era'])
movie_ratings['numVotesQuintiles'] = pd.qcut(movie_ratings['numVotes'], q=5)
movie_ratings.head()

Let's look at a pivot table of averageRating across the bins:

In [ ]:
ratings_by_bin = movie_ratings.pivot_table(index='numVotesQuintiles',
                                           columns='YearGroup',
                                           values='averageRating',
                                           aggfunc='mean')
ratings_by_bin

Same data graphed out:

In [ ]:
ratings_bin_barplot = sns.catplot(data=movie_ratings, 
                                  kind='bar', 
                                  x='YearGroup', 
                                  y='averageRating', 
                                  hue='numVotesQuintiles',
                                  ci=None)
ratings_bin_barplot.set(ylim=(5,8.5), 
                        title='Average movie rating by era and number of votes')

It seems clear that, although there is a positive correlation between number of votes and average rating, the premodern era still has systematically higher ratings across the board, while the IMDB era -- with many more movies being rated -- has lower average ratings.